In [ ]:
%cd ../..
%load_ext autoreload

%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches

from emv.client.get_content import get_frame
from emv.features.pose import load_poses
from emv.features.pose_utils import draw_pose, compare_pose, format_keypoints_to_read, keypoint_name_to_id
from emv.features.pose_utils import KEYPOINTS_NAMES, CONNECTIONS, ANGLES_ASSOCIATIONS

# Load extracted poses

In [ ]:
local_poses_path = "data/pose_df.csv"
pose_df = load_poses(local_poses_path, filter_poses={})

# Filter standing poses

In [ ]:
standing_still_angle = {
    "left_elbow": 0.9,
    "right_elbow": 0.9,
    "left_shoulder": 0.15,
    "right_shoulder": 0.15,
    "left_hip": 0.95,
    "right_hip": 0.95,
    "left_knee":0.95,
    "right_knee":0.95,
    "neck":0.5
}

sitting_pose_angle = {
    "left_elbow": 0.6,
    "right_elbow": 0.6,
    "left_shoulder": 0.15,
    "right_shoulder": 0.15,
    "left_hip": 0.5,
    "right_hip": 0.5,
    "left_knee":0.5,
    "right_knee":0.5,
    "neck":0.5
}

In [ ]:
pose_df["drop"] = pose_df.apply(lambda df: drop_pose(df, sitting_pose_angle, drop_threshold=0.2), axis = 1)
pose_df["drop"].value_counts()

In [ ]:
# create a grid of subplots
nrows = 2
ncols = 6
fig, axs = plt.subplots(nrows=2 * nrows, ncols=ncols, figsize=(ncols * 3, 2 * nrows * 3))
axs = axs.flatten()
# iterate over the poses and draw each pose in a subplot
sample_dropped_poses = pose_df[pose_df["drop"]].sample(nrows * ncols)
sample_kept_poses = pose_df[~pose_df["drop"]].sample(nrows * ncols)
sample_poses = pd.concat([sample_dropped_poses, sample_kept_poses]).reset_index(drop=True)
for i, pose in sample_poses.iterrows():
    draw_pose(pose, ax = axs[i], cut = True)
    axs[i].set_title("drop" if pose["drop"] else "keep")
plt.show()

In [ ]:
for sport in pose_df.sport.unique():
    sport_dropped_poses = pose_df[pose_df["drop"] & (pose_df["sport"] == sport)]
    sport_dropped_poses = sport_dropped_poses.sample(np.min([sport_dropped_poses.shape[0], 10])).reset_index(drop=True)
    if sport_dropped_poses.shape[0] == 0:
        continue
    fig, axs = plt.subplots(1, sport_dropped_poses.shape[0], figsize=(sport_dropped_poses.shape[0] * 3, 3))
    if sport_dropped_poses.shape[0] == 1:
        axs = [axs]
    else:
        axs = axs.flatten()

    for i, pose in sport_dropped_poses.iterrows():
        draw_pose(pose, ax = axs[i], cut = True)
        axs[i].set_title(sport)
    plt.show()

# Filter poses based on bbox

In [ ]:
pose_df[pose_df.media_id == "ioc-SEQ42781636"].to_dict(orient = "records")

In [ ]:
def draw_poses_on_frame(media_data, ax = None, threshold = 0.1):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6,6))

    frame = get_frame(media_data[0]["media_id"], media_data[0]["frame_number"])
    if frame is None:
        return ax
    ax.imshow(frame)

    for pose in media_data:
        keypoints = pose["keypoints"]
        ax.scatter([k[0] for k in keypoints if k[2] > threshold], 
                [k[1] for k in keypoints if k[2] > threshold], 
                s=10)
        for c in CONNECTIONS:
            k1 = keypoints[KEYPOINTS_NAMES.index(c[0])]
            k2 = keypoints[KEYPOINTS_NAMES.index(c[1])]
            if k1[2] > threshold and k2[2] > threshold:
                ax.plot([k1[0], k2[0]], 
                        [k1[1], k2[1]], 
                        linewidth=1, color='black')

        bbox = pose["bbox"]
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)                                              
    
    ax.axis("off")
    ax.set_aspect('equal')
    plt.tight_layout()

    return ax

In [ ]:
draw_poses_on_frame(pose_df[pose_df.media_id == "ioc-SEQ42781636"].to_dict(orient = "records"))

In [ ]:
import requests
import random
from emv.features.pose import process_video, PifPafModel
from emv.client.get_content import authenticate
from emv.settings import API_BASE_URL

MODEL = PifPafModel.fast

In [ ]:
headers = authenticate()

media_id = "ioc-SEQ42781636"
response = requests.get(f"{API_BASE_URL}/download/{media_id}", 
                        headers=headers, 
                        verify=False)
print(f"{API_BASE_URL}/download/{media_id}")
if response.status_code != 200:
    print("Download failed!")

video_bytes = response.content

In [ ]:
frames = process_video(model_name=MODEL,
                       video_bytes = video_bytes,
                       skip_frame = 1)

In [ ]:
len(frames)

In [ ]:
def plot_bbox(media_id, video_bytes = None, skip_frame = 1):
    if video_bytes is None:
        headers = authenticate()
        response = requests.get(f"{API_BASE_URL}/download/{media_id}", 
                                headers=headers, 
                                verify=False)
        print(f"{API_BASE_URL}/download/{media_id}")
        if response.status_code != 200:
            print("Download failed!")

        video_bytes = response.content

    frames = process_video(model_name=MODEL,
                           video_bytes = video_bytes,
                           skip_frame = skip_frame)

    frame = get_frame(media_id, 10)

    fig, ax = plt.subplots(figsize=(6,6))
    ax.imshow(frame)

    for frame in frames:
        frame_data = frame["data"]["annotations"]
        for pose in frame_data:
            bbox = pose["bbox"]
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=0.5, edgecolor='r', facecolor='none', alpha = 0.5)
            ax.add_patch(rect) 

    ax.axis("off")
    plt.tight_layout()
    plt.title(media_id)
    plt.show()

In [ ]:
sample_ids = random.sample(pose_df.media_id.unique().tolist(), 10)

for media_id in sample_ids:
    plot_bbox(media_id, skip_frame = 1)